In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ngsolve import *
from ngsolve.webgui import Draw

In [2]:
import importlib
import sys
from pathlib import Path
import os 
sys.path.append(str(Path(os.getcwd()).parent / "src"))

import active_gel_fem
importlib.reload(active_gel_fem)

import utils 
importlib.reload(utils)

from ngsolve.webgui import Draw

In [3]:
def plot_frame(n_samples, rho, vx, vy, nx, ny, title, stride=3):
    X = np.linspace(0, 1, n_samples)
    Y = np.linspace(0, 1, n_samples)
    X, Y = np.meshgrid(X, Y)

    # Plot the contour for rho_vals
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))

    im0 = axes[0].contourf(X, Y, rho, levels=100, cmap='Greys')
    fig.colorbar(im0, ax=axes[0])

    mag = np.sqrt(vx**2+vy**2)
    im1 = axes[1].contourf(X, Y, mag, levels=100, cmap='Blues')
    axes[1].set_title('v')
    fig.colorbar(im1, ax=axes[1])
    axes[1].quiver(X[::stride, ::stride], Y[::stride, ::stride], 
            (vx/mag)[::stride, ::stride], (vy/mag)[::stride, ::stride], 
            color='orange', 
            width=0.003,     # Increase width to make arrow shafts thicker
            headwidth=4,     # Increase headwidth for wider arrowheads
            headlength=5,    # Increase headlength for longer arrowheads
            headaxislength=7 # Increase headaxislength for more pronounced heads
    ) 

    mag = np.sqrt(nx**2+ny**2)
    im2 = axes[2].contourf(X, Y, mag, levels=100, cmap='Blues')
    axes[2].set_title('nematic order')
    fig.colorbar(im2, ax=axes[2])
    plt.quiver(X[::stride, ::stride], Y[::stride, ::stride], 
            (nx/mag)[::stride, ::stride], (ny/mag)[::stride, ::stride], 
            color='orange', 
            width=0.003,     # Increase width to make arrow shafts thicker
            headlength=0,    
            headaxislength=0 
    ) 

    axes[0].set_title(title)
    plt.tight_layout()
    plt.savefig(f"sim_data/{title}.png", dpi=300)
    plt.show()

def run_sim(eta, beta1, beta2, chi0_active, chi1_active, rho_delta, T, source):
    chi0 = chi0_active
    chi1 = chi1_active

    rho0 = lambda t: 1 + rho_delta*source
    Qsq = lambda t: -1 + 2*source 


    sim = active_gel_fem.NematicActiveGel2D(
                        width=1, height=1, maxh=0.03,
                        gamma=eta, eta_1=eta, eta_2=0,
                        k=1, D=1e-4,
                        kappa = 1e-4,
                        beta1 = beta1, beta2 = beta2,   
                        chi0 = chi0,  chi1 = chi1,
                        Qsq = Qsq, rho0 = rho0 
                    )
    
    # for i in range(len(sim.density.vec.data)): 
    #     sim.density.vec.data[i] = np.random.normal(1, 0.05)
    #     sim.nematic_xx.vec.data[i] = np.random.normal(0, 0.05)

    sim.simulate(
        tend=T,
        tau=0.05,
        save_interval=20 # save once a second 
    )

    return sim 

In [ ]:
# Create simulation instance
T = 100
tau = 0.05
chi0 = 0.01
chi1 = 0.1
beta_prod = 1
beta2 = 5
beta1 = beta_prod/beta2
eta = 1 


tanh = lambda arg: (exp(arg) - exp(-arg))/(exp(arg)+exp(-arg)) 
left_edge = 0.5 - 0.06
right_edge = 0.5 + 0.06
left_transition = 0.5 * (tanh((x - left_edge)/0.01))
right_transition = 0.5 * (tanh((right_edge - x)/0.01))

bottom_transition = 0.5 * (tanh((y - 0.1)/0.1))
top_transition = 0.5 * (tanh((0.9 - y)/0.1))
source =1*(left_transition + right_transition) *(top_transition + bottom_transition)

eta = 1 
for chi0_active in [0.01, 0.03, 0.05]:
    for chi1_active in [0.01, 0.03, 0.05, 0.1, 0.2]:
        if chi1_active > chi0_active: 
            for beta2 in [0.5, 1, 3, 10]:
                for rho_delta in [0.05, 0.08, 0.1]:
                    beta1 = 1/beta2
                    sim = run_sim(eta, beta1, beta2, chi0_active, chi1_active, rho_delta, 100, source)
                    filename = 'sim_data/contractile_everywhere/beta1_{:.2f}_beta2_{:.2f}_chi0_{}_chi1_{}_rhod_{}.npy'.format(beta1, beta2, chi0_active, chi1_active, rho_delta)

                    n_samples = 40 
                    _ = sim.export_to_npy(filename, n_samples=n_samples)

  2%|▏         | 30/2000 [00:00<00:13, 149.76it/s]

craete bilinearformapplication


  2%|▏         | 41/2000 [00:00<00:09, 200.40it/s]

craete bilinearformapplication


  1%|▏         | 28/2000 [00:00<00:13, 141.07it/s]

...

  2%|▎         | 50/2000 [00:00<00:09, 216.10it/s]

craete bilinearformapplication


  3%|▎         | 57/2000 [00:00<00:11, 174.38it/s]

craete bilinearformapplication


100%|██████████| 2000/2000 [00:09<00:00, 213.65it/s]


In [1]:
eta = 1 
for chi0_active in [0.005, 0.01, 0.03, 0.05]:
    for chi1_active in [0.005, 0.01, 0.03, 0.05, 0.1, 0.2]:
        if chi1_active > chi0_active: 
            for beta2 in [0.5, 1, 3, 10]:
                for rho_delta in [0.05, 0.1]:
                    beta1 = 1/beta2
                    filename = 'sim_data/contractile_everywhere/beta1_{:.2f}_beta2_{:.2f}_chi0_{}_chi1_{}_rhod_{}.npy'.format(beta1, beta2, chi0_active, chi1_active, rho_delta)
                    if os.path.isfile(filename):
                        data = np.load(filename)
                        if np.all(np.isfinite(data)):
                            rho, vx, vy, Q, q = np.rollaxis(data[-1], -1)
                            nx, ny = utils.nematic_to_vector(Q, q)

                            plot_frame(
                                n_samples=40, 
                                rho=rho, 
                                vx=vx, vy=vy, 
                                nx=nx, ny=ny,
                                title='contractile_everywhere/beta1_{:.2f}_beta2_{:.2f}_chi0_{}_chi1_{}_rhod_{}'.format(beta1, beta2, chi0_active, chi1_active, rho_delta),
                                stride=3,
                            )

                        
                        


NameError: name 'os' is not defined

In [16]:
sim.visualize(animate=False)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…